# Trabajo Final - Tostadora "Campesino"

## Autor: Leonard David Vivas Dallos

## Objetivo
Predecir la calidad del café basados en el Puntaje de Taza.

## Indicaciones

1. Analizar los datos y definir las variables que se utilizarán de input. Tenga cuidad que no se filtre información al momento de elegirlas.
2. Definir el preprocesamiento que se debe hacer tanto para las X como para la Y.
3. Entrenar al menos dos modelos de regresión.
4. Utilizar un método de explicabilidad (LIME, SHAP, Feature Importance)
5. Reportar resultados.